In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# creating encounters dimesional table

In [0]:
%sql
-- creating a dim_encounter table with external location
CREATE TABLE IF NOT EXISTS gold.dim_encounter (
EncounterID STRING,
SRC_EncounterID STRING,
PatientID STRING,
EncounterDate DATE,
EncounterType STRING,
ProviderID STRING,
DepartmentID STRING,
ProcedureCode INT,
datasource STRING
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_encounter"


In [0]:
%sql
-- reset table
truncate table gold.dim_encounter

In [0]:
%sql
-- inserting into dim_encounter
INSERT INTO gold.dim_encounter
SELECT 
EncounterID,
SRC_EncounterID,
PatientID,
EncounterDate,
EncounterType,
ProviderID,
DepartmentID,
ProcedureCode,
datasource
from silver.encounters
where is_current = true and is_quarantined = false;

In [0]:
%sql
-- getting some records
select * from gold.dim_encounter
where datasource = "hos-b";